In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split

In [3]:
%%time
dataset = pd.read_csv("gpu_data_seawulf.csv")
#dataset.columns = ((dataset.columns.str).replace("^ ","")).str.replace(" $","")
# dataset = dataset[dataset['runtime'].notna()]
dataset = dataset.dropna()
dataset

CPU times: user 233 ms, sys: 45.6 ms, total: 279 ms
Wall time: 291 ms


,Kernel,collapse,Iter,VarDecl,refExpr,intLiteral,floatLiteral,mem_to,mem_from,add_sub_int,add_sub_double,mul_double,div_double,assign_int,assign_double,runtime
0,kernel_100_100_50_50_50_0,0,100,201,2,102,100,0,0,1,0,0,0,0,0,0.056
1,kernel_100_100_50_50_50_0,1,10000,2,0,1,1,0,0,0,0,0,0,0,0,0.001
2,kernel_100_100_50_50_50_1,0,100,201,10598,302,5248,800,800,101,0,0,0,100,5148,0.064
3,kernel_100_100_50_50_50_1,1,10000,2,103,3,51,800,800,1,0,0,0,1,50,0.001
4,kernel_100_100_50_50_50_2,0,100,201,31190,302,100,1600,1600,101,0,5148,0,100,5148,0.066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217019,kernel_6100_2100_3050_3050_1050_29,1,12810000,2,49012,9,3051,97600,97600,4,4100,0,3050,4,7150,28.680
217020,kernel_6100_2100_3050_3050_1050_30,0,6100,4201,128656754,18902,2100,97600,97600,8401,8618296,6410148,6410148,8400,15028444,105.709
217021,kernel_6100_2100_3050_3050_1050_30,1,12810000,2,61212,9,1,97600,97600,4,4100,3050,3050,4,7150,32.684
217022,kernel_6100_2100_3050_3050_1050_31,0,6100,4201,141483350,23102,6412248,97600,97600,10501,8618296,6410148,6410148,10500,21438592,113.709


In [5]:
col_names = ['collapse','Iter','VarDecl','refExpr','intLiteral','floatLiteral','mem_to','mem_from','add_sub_int','add_sub_double','mul_double','div_double','assign_int','assign_double']
features = StandardScaler().fit_transform(dataset[col_names])
runtimes = dataset[['runtime']]
# print(features)
# print(runtimes)

In [6]:
%%time
X_train, X_test, y_train, y_test = train_test_split(features, runtimes, test_size=0.1, random_state=0)
print(f'X_train = {X_train.shape}, X_test = {X_test.shape}, y_train = {y_train.shape}, y_test = {y_test.shape}')
regressor_poly = SVR(kernel='poly').fit(X_train,y_train.values.ravel())

X_train = (194975, 14), X_test = (21664, 14), y_train = (194975, 1), y_test = (21664, 1)
CPU times: user 56min 36s, sys: 12.7 s, total: 56min 49s
Wall time: 1h 32min 32s


In [7]:
%%time
regressor_poly.score(X_test, y_test)

CPU times: user 1min 31s, sys: 350 ms, total: 1min 31s
Wall time: 1min 31s


0.9842005383984519

In [8]:
%%time
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(SVR(kernel='poly'), features, runtimes.values.ravel(), scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score="raise")
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: -29.573 (0.610)
CPU times: user 268 ms, sys: 264 ms, total: 532 ms
Wall time: 2h 18min 32s
